### Movie scripts for Harry Potter, read in as a text file

The scripts are of different formats so good care was taken into examining how would be best to read them into a dataframe for each script individually.

source of data 

https://github.com/asmitakulkarni/QuoteGenerator/tree/master/harrypotterscripts

https://www.kaggle.com/mpwolke/expecto-pythonum/data?select=Harry+Potter+1.csv

Points to fix even more 

 - / is used to merge many characters into one saying the same thing 
 - omitted is used for deleted scenes
 - INT means scene is taken inside 
 - EXT means scene is taken outside,.. this is used to split up the scene id

In [1]:
import pandas as pd
import pickle

Lets print out one of the scripts and see how it looks

In [2]:
file = 'data/Text_script/text7_2_ocr_fixed2.txt'
with open(file) as f:
    L = f.readlines()[:10]

for line in L:
    line = line.strip()
    print(line)

SCABIOR, FENRIR GREYBACK and a ragtag group of SNATCHERS
escort HARRY, RON and HERMIONE past thick yew hedges.
Hermione eyes a white peacock, looking like a ghostly
lawn ornament. Harry, his face horribly swollen WHISPERS:

HARRY
What did you put on me?

HERMIONE
A Stinging Jinx.


In [34]:
# Create function to fix the text and create dataframes if the ocr method is used

# Creating function to clean the text after ocr operation

def ocr_clean(file,file2):
    omitted_endings = [')', '.', '!', '"', ':', '«','?']
    fix_endings = ["(CONT'D)","(0.S.)", "(0O.S.)", "(V.O.)"]
    with open(file) as f:
        L = f.readlines()
        with open(file2, 'w') as outfile:
            prev_empty = False
            prev_character = False
            for line in L:
                line = line.strip()
                first_word = line.split(' ')[0]

                if 'continued' in line.lower() or line.isnumeric() or line.split(" ")[0][:-1].isnumeric() or 'Rev.' in line or 'OMITTED' in line: continue # filter out numbered scenes and continued quotes
                
                if not line: # If the have an empty line
                    if prev_empty == True or prev_character: # check if the previous line we had was also empty or if there was a character before then we skip it
                        continue
                    prev_empty = True

                elif line.isupper() and len(line) > 1: # if we have a character talking
                    for fix in fix_endings:
                        if fix in line:
                            line = line.replace(fix,'')
                            
                    if first_word not in ['INT.','EXT.'] and line[-1] in omitted_endings or '!' in line or '?' in line:
#                         print('lowered lines: ', line)
                        line = line.lower()
                        outfile.write(f'{line}\n')
                        prev_character = False
                        prev_empty = False
                        continue
                    
                    if not prev_empty: # If there was not an empty line before a new character we add it
                        outfile.write(f'\n')
                    prev_empty = False
                    prev_character = True # we dont want empty lines aftur characternames
                else:
                    prev_character = False
                    prev_empty = False

                outfile.write(f'{line}\n')
            
            
# Creating function to make dataframe, takes in the filename and returns a dataframe
def script_df(file,movie_nr,output=False):
    df = pd.DataFrame(columns=['movie_nr','sentence_number','character','sentence','scene_desc','scene_id'])
    # Here we have the scene coming before the characters sentences, the character name comes in with : marker.
    sentence_id = 1 # initialize the sentence counter to 0
    scene_id = 1
    new_scene_id = False
    current_scene = ''
    row_elements = {"movie_nr":movie_nr, "sentence_number": sentence_id, "character": '', "sentence": '', 'scene_desc': "", "scene_id": scene_id}
    new_sentence = False
    same_scene = False
    line_brake = True
    new_character = False
    first = True
    with open(file) as fp: 
        Lines = fp.readlines()
        for line in Lines:
            line = line.strip() # remove \n
            if output:
                print(line)
            if not line: # if not an empty line
                line_brake = True
                new_sentence = False
                continue

            if 'INT.' in line or 'EXT.' in line:
                if output:
                    print('Got new id')
                new_scene_id = True
                continue

            if line.isupper(): # If we have a new character
                if new_scene_id:
                    if output:
                        print('adding to scene_id')
                    scene_id += 1
                    new_scene_id = False
                if output:
                    print('new character')
                new_sentence = True
                line_brake = False
                same_scene = False
                if not first and sentence != "":
                    row_elements['sentence'] = sentence[:-1] # remove the last space
                    row_elements['scene_id'] = scene_id # add the scene id
                    df = df.append(pd.Series(row_elements),ignore_index=True)
                row_elements['character'] = line
                row_elements['sentence_number'] = sentence_id
                sentence_id += 1 
                sentence = ""
                first = False

            elif not line_brake and new_sentence: # same characters sentence
                if output:
                    print('sentence making')
                line_brake = False
                sentence += line + " "

            else:
                line_brake = False
                if same_scene:
                    if output:
                        print('same scene')
                    current_scene += line + " "
                else:
                    if not first:
                        row_elements['sentence'] = sentence[:-1] # remove the last space
                        row_elements['scene_id'] = scene_id # add the scene id
                        df = df.append(pd.Series(row_elements),ignore_index=True)
                        first = True
                    if output:
                        print('new scene')
                    current_scene = line + " "
                row_elements['scene_desc'] = current_scene[:-1] # remove the last space
                same_scene = True
    return(df)

# Movie 1

Character names come with : at the end and this is used to track the sentence each characters says. Here we have a scene: reference and us that to get the scene_id.

In [4]:
# Convert text 1 to dataframe
file = 'data/Text_script/text1.txt'
df1 = pd.DataFrame(columns=['movie_nr','sentence_number','character','sentence','scene_desc','scene_id'])
movie_nr = 1

# Here we have the scene coming before the characters sentences, the character name comes in with : marker.
count = 0
scene_id = 0
character = []
skip_lines = 2
sentence_id = 1 # initialize the sentence counter to 0
current_scene = ''
row_elements = {"movie_nr":1, "sentence_number": '', "character": '', "sentence": '', 'scene_desc': "", 'scene_id': scene_id}
same_scene = True
with open(file) as fp: 
    Lines = fp.readlines()
    for line in Lines:
        line = line.strip()
        if line: # if not an empty line
            first_word = line.split(sep=" ")[0]
            if first_word == "Scene:":
                scene_id += 1
                continue
            elif first_word[-1] != ':': # then we have a a new scene
                if same_scene:
                    row_elements['scene_desc'] = row_elements['scene_desc'] + line
                else:
                    row_elements['scene_desc'] = line
                same_scene = True
            else: # we have a new character sentence
                same_scene = False
                row_elements['sentence_number'] = sentence_id
                row_elements['character'] = first_word[:-1]
                row_elements['sentence'] = line[len(first_word)+1:]
                row_elements['scene_id'] = scene_id
                sentence_id += 1 
            if not same_scene:
#                 print(pd.Series(row_elements))
                df1 = df1.append(pd.Series(row_elements),ignore_index=True)
#                 print(row_elements['sentence'])
df1.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,1,1,Dumbledore,I should have known that you would be here...P...,A neighbourhood on a street called Privet Driv...,1
1,1,2,McGonagall,"Good evening, Professor Dumbledore. Are the ru...","The cat meows, sniffs out and the camera pans ...",1
2,1,3,Dumbledore,"I'm afraid so, Professor. The good, and the bad.","The cat meows, sniffs out and the camera pans ...",1
3,1,4,McGonagall,And the boy?,"The cat meows, sniffs out and the camera pans ...",1
4,1,5,Dumbledore,Hagrid is bringing him.,"The cat meows, sniffs out and the camera pans ...",1


In [5]:
df1[df1.sentence == '']

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id


# Movie 2

Here only the lines are provided and thus it is a little simpler then for script 1, only the characternames ending with : are taken into account and there sentences

In [6]:
# Convert text 2 to dataframe
file = 'data/Text_script/text2.txt'
df2 = pd.DataFrame(columns=['movie_nr','sentence_number','character','sentence','scene_desc','scene_id'])

# Here we have the scene coming before the characters sentences, the character name comes in with : marker.
sentence_id = 1 # initialize the sentence counter to 0
row_elements = {"movie_nr":2, "sentence_number": '', "character": '', "sentence": '', 'scene_desc': "", 'scene_id': scene_id}
same_scene = True
first = True
scene_id = 1
with open(file) as fp: 
    Lines = fp.readlines()
    for line in Lines:
        line = line.strip()
        if line:
            if line.isupper(): # If we have a new character
                if not first:
                    row_elements['sentence'] = sentence[:-1] # remove the last space
                    df2 = df2.append(pd.Series(row_elements),ignore_index=True)
                row_elements['character'] = line
                row_elements['sentence_number'] = sentence_id
                sentence_id += 1
                sentence = ""
                first = False
            else: # we have a new character sentence
                sentence += line + " "
df2.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,2,1,HARRY,"I can't, Hedwig. I'm not allowed to use magic...",,69
1,2,2,UNCLE VERNON,Har-ry Pot-ter!,,69
2,2,3,HARRY,Now you've done it.,,69
3,2,4,UNCLE VERNON,I warned you. If you can't control that blood...,,69
4,2,5,HARRY,She's bored. If I could just let her out for ...,,69


### using the ocr instead: 

In [7]:
file = 'data/Text_script/text2_ocr.txt'
file2 = 'data/Text_script/text2_ocr_fixed.txt'
ocr_clean(file,file2)

df2 = script_df(file2,2) # takes in file path and movie number
df2.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,2,1,HARRY,"I can't, Hedwig. I'm not allowed to use magic ...","Harry pages through a SCRAPBOOK, stops on a MO...",1
1,2,2,UNCLE VERNON,Har-ry Pot-ter!,"At the sound of the name, HEDWIG SQUAWKS again...",1
2,2,3,HARRY,Now you've done it.,"At the sound of the name, HEDWIG SQUAWKS again...",1
3,2,4,UNCLE VERNON,I warned you. If you can't control that bloody...,While AUNT PETUNIA puts the finishing touches ...,2
4,2,5,HARRY,She's bored. If I could just let her out for a...,While AUNT PETUNIA puts the finishing touches ...,2


In [8]:
df2[df2.sentence == '']

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
261,2,262,FATHER'S NOW FACING AN INQUIRY,,Ron looks pale. Clutched in Errol's beak is a ...,30
940,2,941,(1A) 135,,Harry and Ron turn back. See Dumbledore regard...,86


# Movie 3

Same as for script2  only the lines are provided and thus it is a little sipler then for script 1, only the characternames ending with : are taken into account and there sentences

In [9]:
file = 'data/Text_script/text3.txt'
df3 = pd.DataFrame(columns=['movie_nr','sentence_number','character','sentence','scene_desc','scene_id'])

# Here we have the scene coming before the characters sentences, the character name comes in with : marker.
sentence_id = 1 # initialize the sentence counter to 0
row_elements = {"movie_nr":3, "sentence_number": '', "character": '', "sentence": '', 'scene_desc': "", 'scene_id': 1}
same_scene = True
first = True
with open(file) as fp: 
    Lines = fp.readlines()
    for line in Lines:
        line = line.strip()
        if line:
            if line.isupper(): # If we have a new character
                if not first:
                    row_elements['sentence'] = sentence[:-1] # remove the last space
                    df3 = df3.append(pd.Series(row_elements),ignore_index=True)
                row_elements['character'] = line
                row_elements['sentence_number'] = sentence_id
                sentence_id += 1 
                sentence = ""
                first = False
            else: # we have a new character sentence
                sentence += line + " "
df3.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,3,1,HARRY,Lumos Maxima...,,1
1,3,2,HARRY,Lumos Maxima...,,1
2,3,3,HARRY,Lumos... MAXIMA!,,1
3,3,4,AUNT PETUNIA,Harry! Harry!,,1
4,3,5,AUNT PETUNIA,"Well, go on. Open it.",,1


In [10]:
file = 'data/Text_script/text3_ocr.txt'
file2 = 'data/Text_script/text3_ocr_fixed.txt'
ocr_clean(file,file2)       

df3 = script_df(file2,3) # takes in file path and movie number
df3.head(10)

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,3,1,HARRY,"I can't, Hedwig. I'm not allowed to use magic ...","Harry pages through a SCRAPBOOK, stops on a MO...",1
1,3,2,UNCLE VERNON,Har-ry Pot-ter!,"Harry pages through a SCRAPBOOK, stops on a MO...",1
2,3,3,HARRY,Now you've done it.,"Harry pages through a SCRAPBOOK, stops on a MO...",1
3,3,4,UNCLE VERNON,I warned you. If you can't control that bloody...,While AUNT PETUNIA puts the finishing touches ...,2
4,3,5,HARRY,She's bored. If I could just let her out for a...,While AUNT PETUNIA puts the finishing touches ...,2
5,3,6,UNCLE VERNON,And have you sending secret messages to your f...,While AUNT PETUNIA puts the finishing touches ...,2
6,3,7,HARRY,But I haven't gotten any messages. From any of...,While AUNT PETUNIA puts the finishing touches ...,2
7,3,8,DUDLEY,Who'd want to be friends with you?,While AUNT PETUNIA puts the finishing touches ...,2
8,3,9,UNCLE VERNON,I should think you'd be more grateful. We rais...,While AUNT PETUNIA puts the finishing touches ...,2
9,3,10,DUDLEY,I thought he got the second bedroom because Mu...,While AUNT PETUNIA puts the finishing touches ...,2


# Movie 4

This script is a bit more "disassembled", sometimes an empty line comes between mid sentence and thus needed to be tracked.

In [14]:
file = 'data/Text_script/text4.txt'
df4 = pd.DataFrame(columns=['movie_nr','sentence_number','character','sentence','scene_desc'])
movie_nr = 1

# Here we have the scene coming before the characters sentences, the character name comes in with : marker.
sentence_id = 1 # initialize the sentence counter to 0
current_scene = ''
row_elements = {"movie_nr":4, "sentence_number": '', "character": '', "sentence": '', 'scene_desc': ""}
new_sentence = False
same_scene = True
line_brake = True
new_character = False
first = True
with open(file) as fp: 
    Lines = fp.readlines()
    for line in Lines:
        line = line.strip() # remove \n
        # print(line)
        if not line: # if not an empty line
            line_brake = True
            new_sentence = False
            continue
            
#         if not line.isupper() and not new_sentence: continue # skip the scene description
            
        if line.isupper(): # If we have a new character
            # print('new character')
            new_sentence = True
            same_scene = False
            line_brake = False
            if not first:
                row_elements['sentence'] = sentence[:-1] # remove the last space
                df4 = df4.append(pd.Series(row_elements),ignore_index=True)
            row_elements['character'] = line
            row_elements['sentence_number'] = sentence_id
            sentence_id += 1 
            sentence = ""
            first = False
            
        elif not line_brake and new_sentence: # same characters sentence
            # print('sentence making')
            line_brake = False
            sentence += line + " "
            
        else:
            line_brake = False
            if same_scene:
                # print('same scene')
                current_scene += line + " "
            else:
                if not first:
                    row_elements['sentence'] = sentence[:-1] # remove the last space
                    df4 = df4.append(pd.Series(row_elements),ignore_index=True)
                    first = True
                # print('new scene')
                current_scene = line + " "
            row_elements['scene_desc'] = current_scene[:-1] # remove the last space
            same_scene = True

df4.head()

,movie_nr,sentence_number,character,sentence,scene_desc
0,4,1,OLD MAN,Bloody kids.,"An old man lights a gas oven with a match, he ..."
1,4,2,EXT. DARKNESS,,He leaves the house and heads towards the light.
2,4,3,WORMTAIL,Oh no no no my Lord Voldemort. I only meant......,The old man is wandering up to the house carry...
3,4,4,VOLDEMORT,"NO! The boy is everything, it cannot be done w...",The old man is wandering up to the house carry...
4,4,5,UNKNOWN MAN,I will not disappoint you my lord.,The old man is wandering up to the house carry...


In [15]:
file = 'data/Text_script/text4.txt'   

df4 = script_df(file,4) # takes in file path and movie number
df4.head(10)

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,4,1,OLD MAN,Bloody kids.,"An old man lights a gas oven with a match, he ...",1
1,4,2,WORMTAIL,Oh no no no my Lord Voldemort. I only meant......,He leaves the house and heads towards the ligh...,2
2,4,3,VOLDEMORT,"NO! The boy is everything, it cannot be done w...",He leaves the house and heads towards the ligh...,2
3,4,4,UNKNOWN MAN,I will not disappoint you my lord.,He leaves the house and heads towards the ligh...,2
4,4,5,VOLDEMORT,"Good. First, gather our old comrades. Send the...",He leaves the house and heads towards the ligh...,2
5,4,6,VOLDEMORT,Nagini tells me the old caretaker is standing ...,A snake slithers past the old man and into the...,2
6,4,7,VOLDEMORT,Step aside so I can give our guest a proper gr...,Wormtail stands in the doorway and looks at th...,2
7,4,8,HERMIONE,Harry!,A loud spell is cast and Harry wakes up in a f...,3
8,4,9,HERMIONE,Are you alright?,Harry is flustered and still panicing.,3
9,4,10,HARRY,Hermione. Bad dream. When did you get here?,Harry is flustered and still panicing.,3


In [32]:
df4[df4.sentence == '']

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
134,4,135,WEASLEY!,,Mad-Eye throws the chalk in anger.,14
226,4,227,HARRY,,The owl nips Harry's finger.,22
227,4,228,AGH!,,The owl nips Harry's finger.,22
304,4,305,FIVE.,,"Draco calls out, he's sitting up a tree.",26
352,4,353,TWINS,,Harry summons a broom. He jumps on and flies o...,30
353,4,354,NEVER!,,Harry summons a broom. He jumps on and flies o...,30
404,4,405,"ONES WILL HAVE GONE!"".",,"Fred hands Ron a note, it reads ""GET A MOVE ON...",35
436,4,437,ELSEWHERE,,"Soft music plays and everyone is well dressed,...",38
472,4,473,LATER,,Hermione walks off.,39
485,4,486,NIGHTMARE SEQUENCE,,Harry and Ron walk away.,40


# Movie 5

In [17]:
file = 'data/Text_script/text5.txt'
file2 = 'data/Text_script/text5_fixed.txt'

df5 = script_df(file2,5) # takes in file path and movie number
df5.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,5,1,DUDLEY,"Oh, it's you.","It is sunset. We drift through the clouds, app...",1
1,5,2,GANG THUG #1,"What should we do with him, Big D?","It is sunset. We drift through the clouds, app...",1
2,5,3,HARRY,"Big D? Cool name, but to me, you'll always be ...",Harry laughs.,1
3,5,4,DUDLEY,Shut up!,Harry laughs.,1
4,5,5,DUDLEY,"Ooo, tough guy. Not so tough in your bed, are ...",One of Dudley's gang steps in as if to punch H...,1


In [18]:
df5[df5.isna().any(axis=1)]

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id


# Movie 6

In [19]:
# Convert text 2 to dataframe
file = 'data/Text_script/text6_fixed.txt'
df6 = pd.DataFrame(columns=['movie_nr','sentence_number','character','sentence','scene_desc'])

# Here we have the scene coming before the characters sentences, the character name comes in with : marker.
sentence_id = 1 # initialize the sentence counter to 0
row_elements = {"movie_nr":6, "sentence_number": '', "character": '', "sentence": '', 'scene_desc': ""}
same_scene = True
first = True
with open(file) as fp: 
    Lines = fp.readlines()
    for line in Lines:
        line = line.strip()
        if line:
            if line.isupper(): # If we have a new character
                if not first:
                    row_elements['sentence'] = sentence[:-1] # remove the last space
                    df6 = df6.append(pd.Series(row_elements),ignore_index=True)
                row_elements['character'] = line
                row_elements['sentence_number'] = sentence_id
                sentence_id += 1 
                sentence = ""
                first = False
            else: # we have a new character sentence
                sentence += line + " "
df6.head()

,movie_nr,sentence_number,character,sentence,scene_desc
0,6,1,BELLATRIX,Narcissa!,
1,6,2,BELLATRIX,Cissy! You mustn’t do this. He can’t be trusted.,
2,6,3,NARCISSA,The Dark Lord trusts him.,
3,6,4,BELLATRIX,The Dark Lord is mistaken.,
4,6,5,BELLATRIX,Or so I believe.,


In [20]:
file = 'data/Text_script/text6_ocr.txt'
file2 = 'data/Text_script/text6_ocr_fixed.txt'
ocr_clean(file,file2)
        
df6 = script_df(file2,6) # takes in file path and movie number
df6.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,6,1,BELLATRIX,Narcissa!,"The streets of London appear below once more, ...",1
1,6,2,BELLATRIX,Cissy! You mustn’t do this. He can’t be trusted.,"Like a rat in a maze, Narcissa makes her way t...",2
2,6,3,NARCISSA,The Dark Lord trusts him.,"Like a rat in a maze, Narcissa makes her way t...",2
3,6,4,BELLATRIX,The Dark Lord is mistaken.,"Like a rat in a maze, Narcissa makes her way t...",2
4,6,5,BELLATRIX,Or so I believe.,"Narcissa turns, shocked. Bellatrix looks shock...",2


In [21]:
df6[df6.sentence == '']

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
460,6,461,SAME SCENE - LATER,,"Hermione grins, looks up, and gets a WINK from...",39
550,6,551,NEW ANGLE —- HERMIONE - BEHIND THE HANGING —- ...,,"As Neville turns away, Harry notices a pair of...",50
561,6,562,NEW ANGLE - BEHIND THE HANGING —- SAME TIME,,Hermione pops two of the Dragon blobs into her...,50
627,6,628,BACK TO SCENE,,"Similar eerie POV. On the porch, Molly and Art...",54


# Movie 7 (7_1)

In [22]:
# Convert text 2 to dataframe
file = 'data/Text_script/text7.txt'
df7 = pd.DataFrame(columns=['movie_nr','sentence_number','character','sentence','scene_desc'])

# Here we have the scene coming before the characters sentences, the character name comes in with : marker.
sentence_id = 1 # initialize the sentence counter to 0
row_elements = {"movie_nr":7, "sentence_number": '', "character": '', "sentence": '', 'scene_desc': ""}
same_scene = True
first = True
with open(file) as fp: 
    Lines = fp.readlines()
    for line in Lines:
        line = line.strip()
        if line:
            if line.isupper(): # If we have a new character
                if not first:
                    row_elements['sentence'] = sentence[:-1] # remove the last space
                    df7 = df7.append(pd.Series(row_elements),ignore_index=True)
                row_elements['character'] = line
                row_elements['sentence_number'] = sentence_id
                sentence_id += 1 
                sentence = ""
                first = False
            else: # we have a new character sentence
                sentence += line + " "
df7.head()

,movie_nr,sentence_number,character,sentence,scene_desc
0,7,1,SCRIMGEOUR,"These are dark times, there is no denying. Ou...",
1,7,2,MRS. GRANGER,"Hermione. Your tea is ready, dear.",
2,7,3,HERMIONE,"Coming, Mum.",
3,7,4,MRS. GRANGER,Don’t you look lovely. All packed?,
4,7,5,MR. GRANGER,Are you sure these biscuits are sugar-free?,


In [23]:
df7[df7.sentence == '']

,movie_nr,sentence_number,character,sentence,scene_desc
346,7,347,HERMIONE,,
347,7,348,R... A... B.,,
493,7,494,REG CATTERMOLE,,
494,7,495,MARY!,,
600,7,601,HARRY,,
638,7,639,RON,,
639,7,640,NO! YOU DON’T KNOW WHAT IT’S,,
640,7,641,LIKE! YOUR PARENTS ARE DEAD! YOU,,
641,7,642,HAVE NO FAMILY!,,


### Lest try to get the scena also with creating a new ocr of the pdf instead of the given .txt file

In [24]:
file = 'data/Text_script/text7_1_ocr.txt'
file2 = 'data/Text_script/text7_1_ocr_fixed.txt'
ocr_clean(file,file2)

df7 = script_df(file2,7) # takes in file path and movie number
df7.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,7,1,MRS. GRANGER,"Hermione. Your tea is ready, dear.","CAMERA FAVORS a pair of wizards, YAXLEY and PI...",1
1,7,2,HERMIONE,"Coming, Mum.",Hermione glances through the doorway to the st...,1
2,7,3,MRS. GRANGER,Don’t you look lovely. All packed?,"Her mother’s SHADOW withdraws, leaving only th...",2
3,7,4,MR. GRANGER,Are you sure these biscuits are sugar-free?,Hermione nods and watches her mother set the t...,2
4,7,5,MRS. GRANGER,"Quite sure, dear. See the big bold letters tha...",Hermione nods and watches her mother set the t...,2


# Movie 8 (7_2)

Clean the text so we can read it into the dataframe:

In [25]:
file = 'data/Text_script/text7_2_ocr.txt'
file2 = 'data/Text_script/text7_2_ocr_fixed2.txt'
ocr_clean(file,file2)

df8 = script_df(file2,8) # takes in file path and movie number
df8.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,8,1,HARRY,What did you put on me?,"SCABIOR, FENRIR GREYBACK and a ragtag group of...",1
1,8,2,HERMIONE,A Stinging Jinx.,"SCABIOR, FENRIR GREYBACK and a ragtag group of...",1
2,8,3,HARRY,How long will it last?,"SCABIOR, FENRIR GREYBACK and a ragtag group of...",1
3,8,4,HERMIONE,Not long.,"SCABIOR, FENRIR GREYBACK and a ragtag group of...",1
4,8,5,BELLATRIX,Show me.,"Harry glances down, sees his GLASSES cupped in...",1


Read it into the dataframe

In [26]:
# concatenate the dataframes and pickle 
# finished 1, 7_1,7_2
df_total = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)

In [27]:
df_total.head()

,movie_nr,sentence_number,character,sentence,scene_desc,scene_id
0,1,1,Dumbledore,I should have known that you would be here...P...,A neighbourhood on a street called Privet Driv...,1
1,1,2,McGonagall,"Good evening, Professor Dumbledore. Are the ru...","The cat meows, sniffs out and the camera pans ...",1
2,1,3,Dumbledore,"I'm afraid so, Professor. The good, and the bad.","The cat meows, sniffs out and the camera pans ...",1
3,1,4,McGonagall,And the boy?,"The cat meows, sniffs out and the camera pans ...",1
4,1,5,Dumbledore,Hagrid is bringing him.,"The cat meows, sniffs out and the camera pans ...",1


In [28]:
df_total.to_pickle("script_df.pkl")